In [1]:
import pandas as pd

df = pd.read_csv("tactical_dataset_with_results.csv")
print(df)

      possession_ratio  num_passes  avg_pass_length  shots        xg  \
0             0.548188         651        17.239263     20  3.004243   
1             0.451812         521        20.402782      7  0.683733   
2             0.380952         372        23.465847      9  0.545083   
3             0.619048         708        19.188975     20  2.563543   
4             0.547478         566        19.865822     17  1.441534   
...                ...         ...              ...    ...       ...   
6923          0.444366         501        22.856516     11  0.769092   
6924          0.415825         297        24.544687      6  0.550013   
6925          0.584175         501        22.102926     20  1.693702   
6926          0.481178         368        24.493253      4  0.204093   
6927          0.518822         428        22.673465     11  2.174247   

      pressures  tackles  interceptions  avg_x_position  team_width  \
0           220       18              7       63.809874        7